In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import distance

import shapely

In [2]:
df = pd.read_csv('data/all_fix.csv')

In [3]:
def backwar_forward(angle):
    if np.absolute(angle) >= 90:
        return 'b'
    else:
        return 'f'

def upward_downward(angle):
    if angle >= 0:
        return 'u'
    else:
        return 'd'
    
def horizontal_zone(x):
    if x <= 400:
        return 1
    elif x >= 1100:
        return 2
    else:
        return 0

In [4]:
data_frames = []
for p in range(1, 12):
    for c in (1, 2):
        f = df[np.logical_and(df.participant==p, df.condition==c)].reset_index(drop=True)
        f['dx'], f['dy'], f['saccade_length'], f['angularity'] = 0, 0, 0, 0
        
        start = np.array(f[['x_fix', 'y_fix']])[:-1]
        end = np.array(f[['x_fix', 'y_fix']])[1:]
        
        f.loc[1:, 'dx'] = np.absolute(end[:, 0]-start[:, 0])
        f.loc[1:, 'dy'] = np.absolute(end[:, 1]-start[:, 1])

        f.loc[1:, 'saccade_length'] = np.round(np.sqrt((end[:, 0]-start[:, 0])**2 + (end[:, 1]-start[:, 1])**2), 3)
        f.loc[1:, 'angularity'] = np.round(np.arctan2(end[:, 1]-start[:, 1], end[:, 0]-start[:, 0]) * 180 / np.pi, 3)
        f['h_dir'] = f['angularity'].apply(backwar_forward)
        f['v_dir'] = f['angularity'].apply(upward_downward)
        f['zone'] = f.x_fix.apply(horizontal_zone)
        data_frames.append(f)
df = pd.concat(data_frames).reset_index()

In [5]:
df

,index,ts,x,y,event,x_fix,y_fix,duration,doc,participant,condition,dx,dy,saccade_length,angularity,h_dir,v_dir,zone
0,0,86,497,205,Fixation,498.351,203.541,74,D1,1,1,0.000,0.000,0.000,0.000,f,u,0
1,1,92,975,170,Fixation,980.400,172.800,5,D1,1,1,482.049,30.741,483.028,-3.649,f,d,0
2,2,109,1123,167,Fixation,1112.060,176.375,16,D1,1,1,131.660,3.575,131.709,1.555,f,u,2
3,3,126,1290,242,Fixation,1290.000,242.000,8,D1,1,1,177.940,65.625,189.656,20.244,f,u,2
4,4,133,282,227,Fixation,288.667,230.333,6,D1,1,1,1001.333,11.667,1001.401,-179.332,b,d,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8601,451,1535,540,811,Fixation,538.600,805.000,5,D2,11,2,79.219,10.571,79.921,7.601,f,u,0
8602,452,1541,1189,870,Fixation,1184.200,867.600,5,D2,11,2,645.600,62.600,648.628,5.538,f,u,2
8603,453,1554,459,829,Fixation,467.667,810.583,12,D2,11,2,716.533,57.017,718.798,-175.450,b,d,0
8604,454,184,405,793,Fixation,410.714,772.286,7,D2,11,2,56.953,38.297,68.632,-146.082,b,d,0


In [6]:
df['regression'] = False

for i, item in df.iterrows():
    if item.condition == 1:
        thrsh = 64
    else:
        thrsh = 24
    if item.v_dir == 'u' and item.dy > thrsh:
        df.loc[i, 'regression'] = True
    if item.h_dir == 'b' and item.duration >= 7:
        df.loc[i, 'regression'] = True
        if df.loc[i+1, 'h_dir'] == 'f':
            if  item.zone == 1:
                df.loc[i, 'regression'] = False

In [7]:
df['reading'] = 0

f = df[df.regression == False]
f = f[f.dx < 120]
f = f[f.dy < 64]

df.loc[f.index, 'reading'] = 1

In [8]:
df['foreshadow'] = 0

f = df[df.regression == False]
f = f[f.dx >= 120]
f = f[f.dy < 64]

df.loc[f.index, 'foreshadow'] = 1

In [10]:
df.to_csv('fixation_features.csv', sep=',', index=None)